# Exploratory Data Chicago Taxi
_(edited by Brando Palacios)_

## Dataset Chicado Taxi Trips [[1]](#1)

### Resumen
El dataset de estudio es un reporte de viajes en taxi en la ciudad de Chicago, bajo un proceso de regularización y conteo de taxis en la ciudad, todas las empresas afiliadas a estos servicios registran sus taxis mediante un identificador en la _public data city of Chicado_ para proteger la privacidad.


El dataset de Chicado Taxi Trips  fue lanzado desde enero del 2013 hasta la fecha, sin embargo según el reporte de actualizaciones del conjunto de datos de chicado, en octubre del 2017 desactivo la subida de nuevos datos por problemas en la recolección de datos, hasta abril del 2019 que reanudo el reporte de datos. [[2]](#2)

### Justificacion
Sé tomaros los datos de Chicado Taxi por su uso de datos espaciales SQL, los cuales representan información sobre la ubicación física y la forma de los objetos geométricos. Estos objetos pueden ser ubicaciones puntuales u objetos más complejos como cuadrantes, carreteras, etc. [[3]](#3)

In [28]:
import pandas as pd
from dask import dataframe as dd
import dask.multiprocessing
import time

In [51]:
chunk = pd.read_csv("./csv/Taxi_Trips_01012021_07312021.csv")

In [52]:
chunk.head()

,Trip ID,Taxi ID,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,...,Extras,Trip Total,Payment Type,Company,Pickup Centroid Latitude,Pickup Centroid Longitude,Pickup Centroid Location,Dropoff Centroid Latitude,Dropoff Centroid Longitude,Dropoff Centroid Location
0,02fc5d8c3e8525a49b7514bd6c2b3301c2d6aa79,7e179f8ef66ae99ec2d1ec89224e0b7ee5469fe5627f6d...,01/01/2021 12:00:00 AM,01/01/2021 12:15:00 AM,1021.0,3.17,NaN,NaN,NaN,4.0,...,0.0,13.25,Cash,Flash Cab,NaN,NaN,NaN,41.975171,-87.687516,POINT (-87.6875155152 41.9751709433)
1,33eb5c7a1439ac1e3ec6d1520e21db8137d1d11b,2c508057a94474215bdea7b101edad0dc911ef61bb99e6...,01/01/2021 12:00:00 AM,01/01/2021 03:30:00 PM,56047.0,7.10,NaN,NaN,56.0,NaN,...,6.5,27.00,Cash,Flash Cab,41.792592,-87.769615,POINT (-87.7696154528 41.7925923603),NaN,NaN,NaN
2,4bcfcfa78f372b705214ee2cba9c765ad6ba5161,31261f6e7fc645eff98c7964c7ea71a0ea7e387a6bd7f8...,01/01/2021 12:00:00 AM,01/01/2021 12:00:00 AM,120.0,0.10,NaN,NaN,8.0,8.0,...,1.0,4.50,Cash,Taxi Affiliation Services,41.899602,-87.633308,POINT (-87.6333080367 41.899602111),41.899602,-87.633308,POINT (-87.6333080367 41.899602111)
3,655bbd9bcb0a78ee003e304ebbe350b21cdd7083,f6138aa35e1fb074eb79fbe29f512b6153dca3593d8cd2...,01/01/2021 12:00:00 AM,01/01/2021 12:00:00 AM,0.0,0.00,NaN,NaN,67.0,67.0,...,0.0,3.25,Cash,"Taxicab Insurance Agency, LLC",41.775929,-87.666596,POINT (-87.6665962653 41.7759288274),41.775929,-87.666596,POINT (-87.6665962653 41.7759288274)
4,7b7b8108de94a518607068a7369ceea1ab2ae924,2c508057a94474215bdea7b101edad0dc911ef61bb99e6...,01/01/2021 12:00:00 AM,01/01/2021 12:00:00 AM,50.0,0.38,NaN,NaN,60.0,31.0,...,0.0,4.00,Cash,Flash Cab,41.836150,-87.648788,POINT (-87.6487879519 41.8361501547),41.850266,-87.667569,POINT (-87.667569312 41.8502663663)


In [53]:
chunk.shape

(1723696, 23)

Por temas de comodidad hemos descargado los datos particionados por años para luego unirlos en nuestro entorno de trabajo.

In [56]:
# Funcion de union de csv para unificar a un solo CSV
def path_Append(path,chunk):
    chunk2 = pd.read_csv(path)
    chunk = chunk.append(chunk2, ignore_index = True)
    print(chunk.shape)
    del chunk2
    return chunk

In [58]:
chunk = path_Append("./csv/data2.csv",chunk)
chunk = path_Append("./csv/Taxi_Trips_01012019_12312019.csv",chunk)
chunk = path_Append("./csv/Taxi_Trips_01012018_12312018.csv",chunk)
# path_Append("./csv/Taxi_Trips_01012021_07312021.csv",chunk)

(9229703, 23)
(25677152, 23)
(46373279, 23)


### Estructura del Dataset

El dataset posee 23 columnas las cuales describiremos a continuacion:

    
| Columna | Descripción | Tipo de dato |
|:--------:|:---------|---:|
| Trip ID              | Un identificador único para el viaje. | Número|
| Taxi ID              | Un identificador único para el taxi.  | Texto|
| Trip Start Timestamp | Cuando comenzó el viaje, redondeado a los 15 minutos más cercanos. | Timestamp|
| Trip End Timestamp   | Cuando terminó el viaje, redondeado a los 15 minutos más cercanos. | Timestamp|
| Trip Seconds         | Tiempo del viaje en segundos.| Número|
| Trip Miles           | Distancia del viaje en millas.| Número|
| Pickup Census Tract  | La sección censal donde comenzó el viaje. Por privacidad, esta sección censal no se muestra para algunos viajes.  Esta columna a menudo estará en blanco para ubicaciones fuera de Chicago. | Número       |
| Dropoff Census Tract | La sección censal donde terminó el viaje. Por privacidad, esta sección censal no se muestra para algunos viajes. Esta columna a menudo estará en blanco para ubicaciones fuera de Chicago.  | Número       |
| Fare | La tarifa del viaje. | Número       |
| Tips | La propina para el viaje. Las propinas en efectivo generalmente no se registrarán. | Número |
| Tolls | Los peajes para el viaje. | Número |
| Extras | Cargos adicionales por el viaje. | Número |
| Trip Total | Cargos adicionales por el viaje. | Número |
| Payment Type | Tipo de pago del viaje. | Texto |
| Company | La compañía de taxis. | Texto |
| Pickup Centroid Latitude | La latitud del centro de la sección censal de recogida o el área de la comunidad si la sección censal ha sido ocultada por privacidad.  Esta columna a menudo estará en blanco para ubicaciones fuera de Chicago. | Número |
| Pickup Centroid Longitude  | La longitud del centro de la sección censal de recogida o del área de la comunidad si la sección censal ha sido ocultada por privacidad.  Esta columna a menudo estará en blanco para ubicaciones fuera de Chicago. | Número |
| Pickup Centroid Location | La ubicación del centro de la sección censal de recogida o el área de la comunidad si la sección censal se ha ocultado por privacidad.  Esta columna a menudo estará en blanco para ubicaciones fuera de Chicago. | POINT |
| Dropoff Centroid Latitude | La latitud del centro de la sección censal de abandono o el área de la comunidad si la sección censal ha sido ocultada por privacidad.  Esta columna a menudo estará en blanco para ubicaciones fuera de Chicago. | Número |
| Dropoff Centroid Longitude | La longitud del centro de la sección censal de abandono o del área de la comunidad si la sección censal ha sido ocultada por privacidad.  Esta columna a menudo estará en blanco para ubicaciones fuera de Chicago. | Número |
| Dropoff Centroid  Location | La ubicación del centro de la sección censal de entrega o el área de la comunidad si la sección censal ha sido ocultada por privacidad.  Esta columna a menudo estará en blanco para ubicaciones fuera de Chicago. | POINT |

#### Notas del dataset
- En el Área comunitaria (_Área comunitaria_) se encuentra las zonas de los puntos de recojo y destino. Estas zonas estan identificadas en _Chicago Data Portal_ como [__Boundaries Community Areas__](#Boundaries---Community-Areas-(current)--[5])  
- La estructura Point es una estructura del GeoJSON, que identifica la Latitud y Longitud en la Tierra. [[4]](#)

In [59]:
chunk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46373279 entries, 0 to 46373278
Data columns (total 23 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   Trip ID                     object 
 1   Taxi ID                     object 
 2   Trip Start Timestamp        object 
 3   Trip End Timestamp          object 
 4   Trip Seconds                float64
 5   Trip Miles                  float64
 6   Pickup Census Tract         float64
 7   Dropoff Census Tract        float64
 8   Pickup Community Area       float64
 9   Dropoff Community Area      float64
 10  Fare                        float64
 11  Tips                        float64
 12  Tolls                       float64
 13  Extras                      float64
 14  Trip Total                  float64
 15  Payment Type                object 
 16  Company                     object 
 17  Pickup Centroid Latitude    float64
 18  Pickup Centroid Longitude   float64
 19  Pickup Centroid Loc

In [73]:
chunk.shape

(38867272, 23)

In [61]:
chunk = chunk.drop_duplicates(keep='last')

In [64]:
print(f'Se hizo la recoleccion de {chunk.shape[0]} viajes no repetidos')

Se hizo la recoleccion de 38867272 viajes no repetidos


In [65]:
chunk.isnull().sum().sort_values(ascending = False).to_dict()

{'Dropoff Census Tract': 13534348,
 'Pickup Census Tract': 13341054,
 'Tolls': 4458567,
 'Dropoff Community Area': 3754212,
 'Dropoff Centroid  Location': 3619857,
 'Dropoff Centroid Longitude': 3619857,
 'Dropoff Centroid Latitude': 3619857,
 'Pickup Community Area': 2796419,
 'Pickup Centroid Location': 2788783,
 'Pickup Centroid Longitude': 2788783,
 'Pickup Centroid Latitude': 2788783,
 'Trip Seconds': 5879,
 'Trip Total': 3010,
 'Tips': 3010,
 'Extras': 3010,
 'Fare': 3010,
 'Taxi ID': 2067,
 'Trip Miles': 1368,
 'Trip End Timestamp': 1188,
 'Payment Type': 0,
 'Company': 0,
 'Trip Start Timestamp': 0,
 'Trip ID': 0}

In [10]:
chunk = chunk.drop(columns=['Pickup Census Tract','Dropoff Census Tract'])

In [20]:
chunk.columns.tolist()

['Trip ID',
 'Taxi ID',
 'Trip Start Timestamp',
 'Trip End Timestamp',
 'Trip Seconds',
 'Trip Miles',
 'Pickup Community Area',
 'Dropoff Community Area',
 'Fare',
 'Tips',
 'Tolls',
 'Extras',
 'Trip Total',
 'Payment Type',
 'Company',
 'Pickup Centroid Latitude',
 'Pickup Centroid Longitude',
 'Pickup Centroid Location',
 'Dropoff Centroid Latitude',
 'Dropoff Centroid Longitude',
 'Dropoff Centroid  Location']

In [24]:
chunk['Payment Type']

0                 Cash
1                 Cash
2                 Cash
3                 Cash
4                 Cash
              ...     
2623033    Credit Card
2623034    Credit Card
2623035         Mobile
2623036           Cash
2623037    Credit Card
Name: Payment Type, Length: 2623038, dtype: object

In [ ]:
def dropField(col):

In [11]:
chunk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2623038 entries, 0 to 2623037
Data columns (total 21 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   Trip ID                     object 
 1   Taxi ID                     object 
 2   Trip Start Timestamp        object 
 3   Trip End Timestamp          object 
 4   Trip Seconds                float64
 5   Trip Miles                  float64
 6   Pickup Community Area       float64
 7   Dropoff Community Area      float64
 8   Fare                        float64
 9   Tips                        float64
 10  Tolls                       float64
 11  Extras                      float64
 12  Trip Total                  float64
 13  Payment Type                object 
 14  Company                     object 
 15  Pickup Centroid Latitude    float64
 16  Pickup Centroid Longitude   float64
 17  Pickup Centroid Location    object 
 18  Dropoff Centroid Latitude   float64
 19  Dropoff Centroid Long

In [8]:
dtype={'Company': 'object',
       'Dropoff Census Tract': 'float64',
       'Dropoff Community Area': 'float64',
       'Pickup Census Tract': 'float64',
       'Pickup Community Area': 'float64',
       'Trip Seconds': 'float64'}

In [9]:
start = time.time()
dask_df = dd.read_csv('Taxi_Trips_01012018_12312018.csv',blocksize=24e6,dtype=dtype)
end = time.time()
print("Read csv with dask: ",(end-start),"sec")

Read csv with dask:  0.03322863578796387 sec


In [10]:
df = dask_df.compute(scheduler='processes')

## Boundaries - Community Areas (current)  [[5]](#5)

### Resumen
El dataset de estudio es una lista los limites actuales del área comunitaria de Chicago. Estos limites estan representados por varias sub áreas, las cuales representan un sector del mapa de Chicago, el cual no ha cambiado en los ultimos 4 años.
### Justificacion
Se plantea el uso de este dataset para reconocer en que area se encuentran los pedidos, haciendo mas facil el uso de los datos.

In [67]:
comm_area = pd.read_csv("./csv/CommAreas.csv")

In [68]:
comm_area.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   the_geom    77 non-null     object 
 1   PERIMETER   77 non-null     int64  
 2   AREA        77 non-null     int64  
 3   COMAREA_    77 non-null     int64  
 4   COMAREA_ID  77 non-null     int64  
 5   AREA_NUMBE  77 non-null     int64  
 6   COMMUNITY   77 non-null     object 
 7   AREA_NUM_1  77 non-null     int64  
 8   SHAPE_AREA  77 non-null     float64
 9   SHAPE_LEN   77 non-null     float64
dtypes: float64(2), int64(6), object(2)
memory usage: 6.1+ KB


### Estructura del Dataset

El dataset posee 23 columnas las cuales describiremos a continuacion:

    
| Columna | Descripción | Tipo de dato |
|:--------:|:---------|---:|
| the_geom   | Recuadro del área de la comunidad. | MULTIPOLYGON|
| PERIMETER  | Perimetro del área de la comunidad.  | Número|
| AREA       | Area del área de la comunidad. | Número |
| COMAREA_   | Sin Descripcion. | Número |
| COMAREA_ID | Sin Descripcion.| Número |
| AREA_NUMBE | Numero asignado a cada área de comunidad.| Número |
| COMMUNITY  | Nombre de cada área de comunidad. | Texto |
|AREA_NUM_1  | Sin Descripcion.  | Número |
| SHAPE_AREA | Area perimetral del recuadro. | Número |
| SHAPE_LEN  | Distancia perimetral del recuadro. | Número |

#### Notas del dataset
- La estructura __MULTIPOLYGON__ son multiples poligonos en forma de lineas que de conectan para formar un recuadro de varios poligonos, el cual es cerrado. [[3]](#3)

In [70]:
comm_area.head()

,the_geom,PERIMETER,AREA,COMAREA_,COMAREA_ID,AREA_NUMBE,COMMUNITY,AREA_NUM_1,SHAPE_AREA,SHAPE_LEN
0,MULTIPOLYGON (((-87.60914087617894 41.84469250...,0,0,0,0,35,DOUGLAS,35,4.600462e+07,31027.054510
1,MULTIPOLYGON (((-87.59215283879394 41.81692934...,0,0,0,0,36,OAKLAND,36,1.691396e+07,19565.506153
2,MULTIPOLYGON (((-87.62879823733725 41.80189303...,0,0,0,0,37,FULLER PARK,37,1.991670e+07,25339.089750
3,MULTIPOLYGON (((-87.6067081256125 41.816813770...,0,0,0,0,38,GRAND BOULEVARD,38,4.849250e+07,28196.837157
4,MULTIPOLYGON (((-87.59215283879394 41.81692934...,0,0,0,0,39,KENWOOD,39,2.907174e+07,23325.167906


In [69]:
## Eliminaremos algunas columnas que no se utilizaran
comm_area.columns.tolist()

['the_geom',
 'PERIMETER',
 'AREA',
 'COMAREA_',
 'COMAREA_ID',
 'AREA_NUMBE',
 'COMMUNITY',
 'AREA_NUM_1',
 'SHAPE_AREA',
 'SHAPE_LEN']

In [71]:
comm_area.drop(['PERIMETER','AREA','COMAREA_','COMAREA_ID','AREA_NUM_1'], axis=1, inplace=True)

In [72]:
comm_area

,the_geom,AREA_NUMBE,COMMUNITY,SHAPE_AREA,SHAPE_LEN
0,MULTIPOLYGON (((-87.60914087617894 41.84469250...,35,DOUGLAS,4.600462e+07,31027.054510
1,MULTIPOLYGON (((-87.59215283879394 41.81692934...,36,OAKLAND,1.691396e+07,19565.506153
2,MULTIPOLYGON (((-87.62879823733725 41.80189303...,37,FULLER PARK,1.991670e+07,25339.089750
3,MULTIPOLYGON (((-87.6067081256125 41.816813770...,38,GRAND BOULEVARD,4.849250e+07,28196.837157
4,MULTIPOLYGON (((-87.59215283879394 41.81692934...,39,KENWOOD,2.907174e+07,23325.167906
...,...,...,...,...,...
72,MULTIPOLYGON (((-87.69645961375822 41.70714491...,74,MOUNT GREENWOOD,7.558429e+07,48665.130539
73,MULTIPOLYGON (((-87.64215204651398 41.68508211...,75,MORGAN PARK,9.187734e+07,46396.419362
74,MULTIPOLYGON (((-87.83658087874365 41.98639611...,76,OHARE,3.718356e+08,173625.984660
75,MULTIPOLYGON (((-87.65455590025104 41.99816614...,77,EDGEWATER,4.844999e+07,31004.830946


## Bibliography
<a id='1'></a>
[1] _Taxi Trips_, Transportation , Chicago Data Portal, 2021. __URL__:https://data.cityofchicago.org/Transportation/Taxi-Trips/wrvz-psew (visited: 05.12.21)
<br>
<a id='2'></a>
[2] _Update of Chicago Taxi Trips DataSet_, Chicago Data Portal, Developers City of Chicago, 2019. __URL__:http://dev.cityofchicago.org/open%20data/data%20portal/2019/07/01/taxi-dataset-relaunch.html (visited: 05.12.21)
<br>
<a id='3'></a>
[3] _Spatial Data (SQL Server)_, SQL Docs, Microsoft, 2020. __URL__:https://docs.microsoft.com/en-us/sql/relational-databases/spatial/spatial-data-sql-server?view=sql-server-ver15 (visited: 05.12.21)
<br>
<a id='4'></a>
[4] _DataTypes_, API Docs, SODA,2015. __URL__:https://dev.socrata.com/docs/datatypes/ (visited: 05.12.21)
<br>
<a id='5'></a>
[5] _Boundaries - Community Areas (current)_, Chicago Data Portal ,2013. __URL__:https://data.cityofchicago.org/Facilities-Geographic-Boundaries/Boundaries-Community-Areas-current-/cauq-8yn6/data (visited: 05.12.21)